In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,BatchNormalization,Dropout,MaxPooling2D
import numpy as np
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from random import shuffle
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [5]:
from google.colab import files
uploaded = files.upload()

Saving data.zip to data.zip


In [6]:
import zipfile
import os

In [7]:
!unzip /content/data.zip -d /content/

Archive:  /content/data.zip
   creating: /content/data/
   creating: /content/data/rakesh/
  inflating: /content/data/rakesh/1.jpg  
  inflating: /content/data/rakesh/2.jpg  
  inflating: /content/data/rakesh/3.jpg  
  inflating: /content/data/rakesh/4.jpg  
  inflating: /content/data/rakesh/5.jpg  
  inflating: /content/data/rakesh/6.jpg  
  inflating: /content/data/rakesh/7.jpg  
  inflating: /content/data/rakesh/8.jpg  
  inflating: /content/data/rakesh/9.jpg  
  inflating: /content/data/rakesh/10.jpg  
  inflating: /content/data/rakesh/18.jpg  
  inflating: /content/data/rakesh/11.jpg  
  inflating: /content/data/rakesh/12.jpg  
  inflating: /content/data/rakesh/13.jpg  
  inflating: /content/data/rakesh/14.jpg  
  inflating: /content/data/rakesh/15.jpg  
  inflating: /content/data/rakesh/16.jpg  
  inflating: /content/data/rakesh/17.jpg  
  inflating: /content/data/rakesh/19.jpg  
  inflating: /content/data/rakesh/20.jpg  
  inflating: /content/data/rakesh/21.jpg  
  inflating: /c

In [8]:
#create labels
dir = "/content/data"
folders = [entry.name for entry in os.scandir(dir) if entry.is_dir()]
folders = np.unique(folders)
print(folders)

label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(folders)
labels_array = np.array(numerical_labels)
num_classes = len(np.unique(labels_array))
labels = to_categorical(labels_array, num_classes)

['maa' 'rakesh']


In [9]:
labels = zip(folders,labels)
labels = tuple(labels)

In [10]:
labels[0][1]

array([1., 0.], dtype=float32)

In [11]:
def data():
  data = []
  for f in folders:
    dir = f"/content/data/{f}"
    label = [i for i in labels if i[0] == str(f)]
    label = label[0][1]
    for img in os.listdir(dir):
      img_path = os.path.join(dir, img)
      img_data = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
      img_data = cv2.resize(img_data,(256,256))
      data.append([np.array(img_data),label])
  shuffle(data)
  return data
dataset = data()

In [12]:
dataset[0][1]

array([0., 1.], dtype=float32)

In [32]:
images = np.array([item[0] for item in dataset])
labels = np.array([item[1] for item in dataset])

images = images.astype('float32') / 255.0

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [33]:
y_train

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [34]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [35]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis =-1)
train_generator = datagen.flow(X_train, y_train, batch_size=32)
test_generator = datagen.flow(X_test, y_test, batch_size=32)

In [36]:
model = Sequential()

model.add(layers.InputLayer(input_shape=(images.shape[1], images.shape[2],1)))
# Convolutional Block 1
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2)))

# Convolutional Block 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2)))

# Convolutional Block 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2)))

# Fully Connected Block
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(2, activation='softmax'))



In [38]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [39]:
# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 batch_normalization_4 (Bat  (None, 254, 254, 32)      128       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 254, 254, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_5 (Bat  (None, 125, 125, 64)     

In [40]:
history = model.fit(train_generator , epochs=10, batch_size=32,  validation_data=test_generator)

Epoch 1/10
50/50 [==============================] - 18s 281ms/step - loss: 0.0664 - accuracy: 0.9850 - val_loss: 121.5279 - val_accuracy: 0.5325
Epoch 2/10
50/50 [==============================] - 14s 270ms/step - loss: 0.0118 - accuracy: 0.9969 - val_loss: 152.4068 - val_accuracy: 0.5325
Epoch 3/10
50/50 [==============================] - 13s 267ms/step - loss: 0.0223 - accuracy: 0.9956 - val_loss: 129.6393 - val_accuracy: 0.5325
Epoch 4/10
50/50 [==============================] - 13s 266ms/step - loss: 0.0055 - accuracy: 0.9975 - val_loss: 69.0889 - val_accuracy: 0.5325
Epoch 5/10
50/50 [==============================] - 14s 273ms/step - loss: 0.0053 - accuracy: 0.9987 - val_loss: 0.0268 - val_accuracy: 0.9900
Epoch 6/10
50/50 [==============================] - 16s 316ms/step - loss: 5.9179e-04 - accuracy: 1.0000 - val_loss: 1.0843 - val_accuracy: 0.7275
Epoch 7/10
50/50 [==============================] - 14s 273ms/step - loss: 0.0032 - accuracy: 0.9981 - val_loss: 4.2211 - val_accur

In [42]:
model.save('indentify.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
